# ComfyUI + Manager — Optimized Colab Setup

**Performance optimizations:**
- Installs to **local SSD** (not Google Drive) for fast git/pip operations
- Skips redundant PyTorch install (Colab already has it)
- Single combined pip install instead of 5 separate calls
- Shallow git clones (`--depth 1`)
- Pip cache persisted on Google Drive for faster repeat sessions
- Custom nodes backup/restore via tarball on Drive
- **Model dirs symlinked to Google Drive** — downloads via Manager go directly to Drive and persist across sessions

**First run:** ~10-15 min | **Repeat runs (with backup):** ~2-5 min

In [ ]:
#@title Environment Setup
import os, shutil, time

# ============================================================
# Configuration
# ============================================================
USE_GOOGLE_DRIVE = True          #@param {type:"boolean"}
UPDATE_COMFY_UI = True           #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True       #@param {type:"boolean"}
RESTORE_CUSTOM_NODES = True      #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPS = True #@param {type:"boolean"}

# Paths — edit these to match your Drive layout
DRIVE_MODELS_ROOT = "/content/drive/MyDrive/AI/models"
DRIVE_COMFYUI_PERSIST = "/content/drive/MyDrive/AI/models/COMFYUI"
LOCAL_WORKSPACE = "/content/ComfyUI"   # local SSD — fast!
PIP_CACHE_DIR = "/content/drive/MyDrive/AI/.pip-cache"

_start = time.time()

# ============================================================
# 1. Mount Google Drive & set pip cache
# ============================================================
if USE_GOOGLE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    os.makedirs(PIP_CACHE_DIR, exist_ok=True)
    os.environ['PIP_CACHE_DIR'] = PIP_CACHE_DIR
    os.makedirs(DRIVE_COMFYUI_PERSIST, exist_ok=True)

# ============================================================
# 2. Load Colab secrets (HF, CivitAI, ngrok)
# ============================================================
try:
    from google.colab import userdata
    _hf = userdata.get('HF_TOKEN')
    if _hf:
        os.environ['HF_TOKEN'] = _hf
        os.environ['HUGGING_FACE_HUB_TOKEN'] = _hf
        print("HF_TOKEN loaded")
    _civit = userdata.get('CIVIT_TOKEN')
    if _civit:
        os.environ['CIVIT_TOKEN'] = _civit
        print("CIVIT_TOKEN loaded")
    _ngrok = userdata.get('NGROK_AUTHTOKEN')
    if _ngrok:
        os.environ['NGROK_AUTHTOKEN'] = _ngrok
        print("NGROK_AUTHTOKEN loaded")
except Exception:
    print("Could not load Colab secrets — set them in Colab sidebar > Secrets")

# ============================================================
# 3. Clone / update ComfyUI on local SSD (fast!)
# ============================================================
if not os.path.isdir(LOCAL_WORKSPACE):
    print("=== Cloning ComfyUI (shallow) ===")
    !git clone --depth 1 https://github.com/comfyanonymous/ComfyUI {LOCAL_WORKSPACE}
elif UPDATE_COMFY_UI:
    print("=== Updating ComfyUI ===")
    !cd {LOCAL_WORKSPACE} && git pull

%cd {LOCAL_WORKSPACE}

# Set COMFYUI_PATH so cm-cli.py knows where ComfyUI lives
os.environ['COMFYUI_PATH'] = LOCAL_WORKSPACE

# ============================================================
# 3b. Symlink model dirs → Google Drive (downloads go to Drive!)
# ============================================================
# This replaces default model dirs with symlinks so that ALL
# model downloads (via Manager install, custom nodes, etc.)
# automatically land on Google Drive and persist across sessions.
MODEL_SYMLINKS = {
    'checkpoints':      f'{DRIVE_MODELS_ROOT}/checkpoints',
    'vae':              f'{DRIVE_MODELS_ROOT}/vae',
    'loras':            f'{DRIVE_MODELS_ROOT}/loras',
    'controlnet':       f'{DRIVE_MODELS_ROOT}/controlnet',
    'clip':             f'{DRIVE_MODELS_ROOT}/clip',
    'embeddings':       f'{DRIVE_MODELS_ROOT}/embeddings',
    'gligen':           f'{DRIVE_MODELS_ROOT}/gligen',
    'upscale_models':   f'{DRIVE_MODELS_ROOT}/upscale_models',
    'style_models':     f'{DRIVE_MODELS_ROOT}/style_models',
    'diffusion_models': f'{DRIVE_MODELS_ROOT}/diffusion_base',
    'text_encoders':    f'{DRIVE_MODELS_ROOT}/text_encoders',
}

if USE_GOOGLE_DRIVE:
    print("=== Linking model dirs to Google Drive ===")
    models_dir = os.path.join(LOCAL_WORKSPACE, 'models')
    for folder, drive_target in MODEL_SYMLINKS.items():
        local_path = os.path.join(models_dir, folder)
        os.makedirs(drive_target, exist_ok=True)
        if os.path.islink(local_path):
            os.remove(local_path)
        elif os.path.isdir(local_path):
            shutil.rmtree(local_path)
        os.symlink(drive_target, local_path)
        print(f"  {folder}/ -> {drive_target}")

# ============================================================
# 4. Install Python dependencies
# ============================================================
# Colab already has PyTorch + CUDA — install everything else from
# ComfyUI's own requirements.txt so we stay current with new deps.
print("=== Installing ComfyUI dependencies ===")
!pip install -q -r requirements.txt GitPython

# ============================================================
# 5. Install ComfyUI-Manager on local SSD
# ============================================================
if USE_COMFYUI_MANAGER:
    _mgr = os.path.join(LOCAL_WORKSPACE, "custom_nodes", "ComfyUI-Manager")
    if not os.path.isdir(_mgr):
        print("=== Cloning ComfyUI-Manager (shallow) ===")
        !git clone --depth 1 https://github.com/ltdrdata/ComfyUI-Manager {_mgr}
    elif UPDATE_COMFY_UI:
        !cd {_mgr} && git pull

# ============================================================
# 6. Restore custom nodes backup from Drive (if available)
# ============================================================
_backup = os.path.join(DRIVE_COMFYUI_PERSIST, "custom_nodes_backup.tar.gz")
_custom_dir = os.path.join(LOCAL_WORKSPACE, "custom_nodes")
if RESTORE_CUSTOM_NODES and os.path.isfile(_backup):
    print("=== Restoring custom nodes from Drive backup ===")
    !cd {LOCAL_WORKSPACE} && tar xzf {_backup}
    print(f"Restored from {_backup}")
elif RESTORE_CUSTOM_NODES:
    print("No custom nodes backup found — will start fresh.")
    print(f"  (Run the 'Save Custom Nodes Backup' cell later to create one)")

# ============================================================
# 7. Install custom node dependencies
# ============================================================
if INSTALL_CUSTOM_NODES_DEPS and USE_COMFYUI_MANAGER:
    print("=== Installing custom node dependencies ===")
    !cd {LOCAL_WORKSPACE} && python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies

# ============================================================
# 8. Write extra_model_paths.yaml (non-standard types only)
# ============================================================
# Standard model types (checkpoints, vae, loras, etc.) are handled
# by symlinks above. This YAML only covers custom types that don't
# have a default ComfyUI model directory.
_yaml_path = os.path.join(LOCAL_WORKSPACE, "extra_model_paths.yaml")
_yaml_content = f"""# Auto-generated — standard model dirs use symlinks to Drive
# This file only covers non-standard model types
google_drive:
    base_path: {DRIVE_MODELS_ROOT}/
    LLM: llm/
    audio: audio_models/
"""
with open(_yaml_path, 'w') as f:
    f.write(_yaml_content)
print(f"Wrote {_yaml_path}")

# ============================================================
# Done!
# ============================================================
_elapsed = time.time() - _start
print(f"\n{'='*50}")
print(f"Setup complete in {_elapsed:.0f}s ({_elapsed/60:.1f} min)")
print(f"Workspace: {LOCAL_WORKSPACE}")
print(f"Models:    {DRIVE_MODELS_ROOT}/ (via symlinks)")
print(f"{'='*50}")

### Download models to Google Drive (optional)
Models are saved to your Drive so they persist across sessions. Uses your HF_TOKEN and CIVIT_TOKEN for gated/authenticated downloads. Uncomment the models you want.

In [ ]:
import os

MODELS = "/content/drive/MyDrive/AI/models"
HF_AUTH = f'--header="Authorization: Bearer {os.environ.get("HF_TOKEN", "")}"' if os.environ.get("HF_TOKEN") else ""
CIVIT_TOKEN = os.environ.get("CIVIT_TOKEN", "")

# ---- Checkpoints ----

# SD1.5
#!wget -c {HF_AUTH} https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -P {MODELS}/checkpoints/

# SDXL (recommended workflows: https://comfyanonymous.github.io/ComfyUI_examples/sdxl/)
#!wget -c {HF_AUTH} https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors -P {MODELS}/checkpoints/
#!wget -c {HF_AUTH} https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors -P {MODELS}/checkpoints/

# SD2
#!wget -c {HF_AUTH} https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors -P {MODELS}/checkpoints/
#!wget -c {HF_AUTH} https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors -P {MODELS}/checkpoints/

# SDXL ReVision
#!wget -c {HF_AUTH} https://huggingface.co/comfyanonymous/clip_vision_g/resolve/main/clip_vision_g.safetensors -P {MODELS}/clip/

# ---- VAE ----
#!wget -c {HF_AUTH} https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P {MODELS}/vae/

# ---- LoRAs ----
#!wget -c {HF_AUTH} https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors -P {MODELS}/loras/

# CivitAI LoRAs (uses your CIVIT_TOKEN)
#!wget -c --content-disposition "https://civitai.com/api/download/models/10350?token={CIVIT_TOKEN}" -P {MODELS}/loras/
#!wget -c --content-disposition "https://civitai.com/api/download/models/10638?token={CIVIT_TOKEN}" -P {MODELS}/loras/

# ---- ControlNet ----
#!wget -c {HF_AUTH} https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -P {MODELS}/controlnet/
#!wget -c {HF_AUTH} https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -P {MODELS}/controlnet/
#!wget -c {HF_AUTH} https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -P {MODELS}/controlnet/
#!wget -c {HF_AUTH} https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -P {MODELS}/controlnet/
#!wget -c {HF_AUTH} https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -P {MODELS}/controlnet/

# ControlNet SDXL
#!wget -c {HF_AUTH} https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-canny-rank256.safetensors -P {MODELS}/controlnet/
#!wget -c {HF_AUTH} https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-depth-rank256.safetensors -P {MODELS}/controlnet/

# ---- GLIGEN ----
#!wget -c {HF_AUTH} https://huggingface.co/comfyanonymous/GLIGEN_pruned_safetensors/resolve/main/gligen_sd14_textbox_pruned_fp16.safetensors -P {MODELS}/gligen/

# ---- Upscale models ----
#!wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P {MODELS}/upscale_models/
#!wget -c {HF_AUTH} https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -P {MODELS}/upscale_models/
#!wget -c {HF_AUTH} https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -P {MODELS}/upscale_models/

# ---- Style models ----
#!wget -c {HF_AUTH} https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -P {MODELS}/style_models/

# ---- CLIP Vision ----
#!wget -c {HF_AUTH} https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin -O {MODELS}/clip/clip_vit14.bin

print("Model downloads complete.")

### Save Custom Nodes Backup to Google Drive
Run this cell **after installing new custom nodes** via the Manager UI to persist them across Colab sessions. The backup is a single tarball — much faster to restore than re-cloning everything.

In [ ]:
import os, time

LOCAL_WORKSPACE = "/content/ComfyUI"
DRIVE_COMFYUI_PERSIST = "/content/drive/MyDrive/AI/models/COMFYUI"
_backup = os.path.join(DRIVE_COMFYUI_PERSIST, "custom_nodes_backup.tar.gz")

os.makedirs(DRIVE_COMFYUI_PERSIST, exist_ok=True)

print("=== Saving custom nodes backup to Google Drive ===")
_t = time.time()
!cd {LOCAL_WORKSPACE} && tar czf {_backup} custom_nodes/
_sz = os.path.getsize(_backup) / (1024*1024)
print(f"Saved {_sz:.0f} MB to {_backup} in {time.time()-_t:.0f}s")
print("This backup will be restored automatically on your next session.")

### Run ComfyUI with ngrok (Recommended)
Requires `NGROK_AUTHTOKEN` in Colab Secrets. Get a free token at [ngrok.com](https://dashboard.ngrok.com/signup). More reliable than cloudflared and gives you a stable URL.

In [ ]:
import os, subprocess, threading, time, socket

!pip install -q pyngrok

def ngrok_tunnel(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    from pyngrok import ngrok
    token = os.environ.get('NGROK_AUTHTOKEN', '')
    if not token:
        print("ERROR: NGROK_AUTHTOKEN not set. Add it to Colab Secrets.")
        return
    ngrok.set_auth_token(token)
    tunnel = ngrok.connect(port)
    print(f"\nComfyUI is ready! Access it at: {tunnel.public_url}\n")

threading.Thread(target=ngrok_tunnel, daemon=True, args=(8188,)).start()

!cd /content/ComfyUI && python main.py --dont-print-server

### Run ComfyUI with cloudflared (Alternative)
Use this if you don't have an ngrok account. No signup required — uses temporary Cloudflare tunnel URLs.

In [ ]:
!wget -q -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb > /dev/null 2>&1

import subprocess, threading, time, socket

def cloudflared_tunnel(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    print("\nComfyUI finished loading, launching cloudflared tunnel...\n")
    p = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE
    )
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
            print("ComfyUI URL:", l[l.find("http"):], end='')

threading.Thread(target=cloudflared_tunnel, daemon=True, args=(8188,)).start()

!cd /content/ComfyUI && python main.py --dont-print-server

### Run ComfyUI with localtunnel (Alternative)

In [ ]:
!npm install -g localtunnel

import subprocess, threading, time, socket, urllib.request

def localtunnel_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    print("\nComfyUI finished loading, launching localtunnel...\n")
    print("The password/endpoint ip for localtunnel is:",
          urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
    p = subprocess.Popen(["lt", "--port", str(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
        print(line.decode(), end='')

threading.Thread(target=localtunnel_thread, daemon=True, args=(8188,)).start()

!cd /content/ComfyUI && python main.py --dont-print-server

### Run ComfyUI with Colab iframe (Fallback)
Use only if tunnels above don't work. Note: live image previews won't work because the Colab iframe blocks WebSockets.

In [ ]:
import threading, time, socket

def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    from google.colab import output
    output.serve_kernel_port_as_iframe(port, height=1024)
    print("To open in a separate window:")
    output.serve_kernel_port_as_window(port)

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!cd /content/ComfyUI && python main.py --dont-print-server